In [8]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SQLContext
import json
import math
import arrow
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("cohort") \
    .config("spark.some.config.option", "cohort") \
    .getOrCreate()

def getDatetimeFromEpoch(epoch=None):
    if epoch is not None:
        epoch = epoch/1000
        utc_date = arrow.get(epoch)
        local = utc_date.to('Asia/Kolkata')
        local = local.datetime
        return local
    else:
        return None
    
def getCohortGroupId(action_time):
    return "week" + str(getWeekNumberFromDateTime(action_time))

def getWeekNumberFromDateTime(action_time):
    return getDatetimeFromEpoch(action_time).strftime("%V")

def removeUtmParam(utmString):
    return utmString.replace("utm_source:", "")

def week22(row):
    return "week22" == row[1]

def week23(row):
    return "week23" == row[1]

def week24(row):
    return "week24" == row[1]

def week25(row):
    return "week25" == row[1]

def week26(row):
    return "week26" == row[1]

cohort_map = {"Criteo":{}, "google":{}, "Facebook":{}, "Email":{}, "sms":{}}
file_path = "/home/marino/Desktop/website_events.csv"
df  = spark.read.format("csv").option("header", "true").load(file_path)
track_events = df.where(df["eventType"] == "UTM_PARAMS")
view_events = df.where(df["eventType"] == "PRODUCT_VIEW")
df1 = track_events.alias('df1')
df2 = view_events.alias('df2')
track_plus_view_events = df1.join(df2, df1.userId == df2.userId).select(['df1.productId', 'df2.userId', 'df2.eventTime'])
overall_view_actions = view_events.rdd.map((lambda x: (x["userId"], getCohortGroupId(int(x["eventTime"])))))
cohort = track_plus_view_events.rdd.filter((lambda x: "utm_source" in x["productId"])).map((lambda x: (removeUtmParam(x["productId"]), x["userId"], getCohortGroupId(int(x["eventTime"])))))
cohort_df = cohort.toDF(["source", "userId", "cohortId"])
cohort_df.write.partitionBy("source").csv("cohorts")

sources =  [ "google", "Facebook", "Email", "sms"] 
for source in sources:
    print 'loading ' + source +  " cohort "
    cohortGroup  = spark.read.format("csv").load("cohorts/source=" + source)
    cohortGroup_rdd = cohortGroup.rdd.map(lambda x: (x._c0, x._c1))
    week22_rdd, week23_rdd, week24_rdd, week25_rdd, week26_rdd = (cohortGroup_rdd.filter(f).map(lambda f: f[0]).distinct() for f in (week22, week23, week24, week25, week26))
    cohort_size = week22_rdd.count() 
    print "Cohort size", cohort_size
    cohort_map[source]["week0"] = 100
    cohort_map[source]["week1"] = (week22_rdd.intersection(week23_rdd).count() * 100 ) / cohort_size
    cohort_map[source]["week2"] = (week22_rdd.intersection(week24_rdd).count() * 100) / cohort_size
    cohort_map[source]["week3"] = (week22_rdd.intersection(week25_rdd).count() * 100) / cohort_size
    cohort_map[source]["week4"] = (week22_rdd.intersection(week26_rdd).count()* 100 ) / cohort_size

AnalysisException: u'path file:/home/marino/spark/cohorts already exists.;'

In [9]:

print cohort_map

loading sms cohort 
Cohort size 182
{'sms': {'week1': 59, 'week0': 100, 'week3': 38, 'week2': 51, 'week4': 45}, 'Criteo': {}, 'google': {}, 'Facebook': {}, 'Email': {}}
